In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

In [2]:
# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# define a model
model = Sequential([
    Flatten(input_shape = (28,28)),
    Dense(128, activation = 'relu'),
    Dense(10)
])

c:\Users\ghummatova\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# define loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

In [5]:
# implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.552339553833008
Epoch 1 Step 200: Loss = 0.4023306965827942
Epoch 1 Step 400: Loss = 0.18397483229637146
Epoch 1 Step 600: Loss = 0.18389032781124115
Epoch 1 Step 800: Loss = 0.1440712809562683
Epoch 1 Step 1000: Loss = 0.4658580422401428
Epoch 1 Step 1200: Loss = 0.18269610404968262
Epoch 1 Step 1400: Loss = 0.23084281384944916
Epoch 1 Step 1600: Loss = 0.20811232924461365
Epoch 1 Step 1800: Loss = 0.12597188353538513
Start of epoch 2
Epoch 2 Step 0: Loss = 0.08379092812538147
Epoch 2 Step 200: Loss = 0.14957763254642487
Epoch 2 Step 400: Loss = 0.11971994489431381
Epoch 2 Step 600: Loss = 0.071737140417099
Epoch 2 Step 800: Loss = 0.06520923972129822
Epoch 2 Step 1000: Loss = 0.24964426457881927
Epoch 2 Step 1200: Loss = 0.10491427034139633
Epoch 2 Step 1400: Loss = 0.1583489179611206
Epoch 2 Step 1600: Loss = 0.1695874035358429
Epoch 2 Step 1800: Loss = 0.06552714854478836


In [6]:
# enhance the custom training loop by adding an accuracy metric to monitor model performance

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


In [7]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 0.04134659469127655 Accuracy = 1.0
Epoch 1 Step 200: Loss = 0.05511271208524704 Accuracy = 0.9735696315765381
Epoch 1 Step 400: Loss = 0.10105249285697937 Accuracy = 0.9724127054214478
Epoch 1 Step 600: Loss = 0.04928106814622879 Accuracy = 0.9734296798706055
Epoch 1 Step 800: Loss = 0.03739263489842415 Accuracy = 0.973665714263916
Epoch 1 Step 1000: Loss = 0.11521053314208984 Accuracy = 0.9741820693016052
Epoch 1 Step 1200: Loss = 0.04165233299136162 Accuracy = 0.9746044874191284
Epoch 1 Step 1400: Loss = 0.1316077560186386 Accuracy = 0.9750401377677917
Epoch 1 Step 1600: Loss = 0.12712788581848145 Accuracy = 0.9746837615966797
Epoch 1 Step 1800: Loss = 0.0488007627427578 Accuracy = 0.9751526713371277
Start of epoch 2
Epoch 2 Step 0: Loss = 0.016839366406202316 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.06743358075618744 Accuracy = 0.981809675693512
Epoch 2 Step 400: Loss = 0.07800669968128204 Accuracy = 0.9810629487037659
Epoch 2 Step 600: Loss 

In [8]:
# Custom callback for advanced logging
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')
